In [ ]:
import torch
import numpy as np
from sklearn import datasets
from matplotlib import pyplot as plt

# keeps double precision
torch.set_default_tensor_type(torch.DoubleTensor)

In [7]:
source = datasets.load_digits()
data = source.data
target = source.target

In [62]:
data = source.data
target = source.target

features = 64
classes = 10

design = np.insert(data, 0, 1., 1)
onehot = (np.arange(classes) == target[:, None])

# the default torch precision is float32 and int64 respectively
DESIGN = torch.tensor(design)#, dtype = torch.float32)
TARGET = torch.tensor(target)#, dtype = torch.int64)
ONEHOT = torch.tensor(onehot)

print(design.dtype, target.dtype, onehot.dtype)
print(DESIGN.dtype, TARGET.dtype, ONEHOT.dtype)

float64 int64 bool
torch.float64 torch.int64 torch.bool


In [83]:
def torch_activity( design, param ):
    ACTIVATION = design @ param
    EXP = torch.exp(ACTIVATION)
    SUM = torch.sum(EXP, 1, keepdim = True)
    return EXP/SUM

In [84]:
PARAM = torch.zeros(1+ features, classes, requires_grad = True)#, dtype=torch.float64)
scaler = torch.cuda.amp.GradScaler()

opt = torch.optim.SGD([PARAM], lr = 10**(-4))
for epoch in range(1000):
    
    ACTIVITY = torch_activity( DESIGN, PARAM )
    LOG = torch.log(ACTIVITY)
    ENTROPY = - LOG * ONEHOT
    LOSS = torch.sum(ENTROPY)
        
    LOSS.backward()
    opt.step()
    opt.zero_grad()
    
    print(epoch, LOSS.item())

0 4137.745412110294
1 649.7969263193856
2 12075.460247977806
3 70934.74638089178
4 113185.81136619221
5 147726.24472165358
6 124791.24901414828
7 120991.2350140429
8 95682.53428322486
9 77140.97009934513
10 90179.61212248428
11 65123.17523275957
12 32889.2979099294
13 15180.826275339019
14 16698.458256596725
15 3401.9373175946407
16 2645.972631245703
17 2512.196438212264
18 3949.5005461133114
19 3605.1172807260723
20 8041.159174494795
21 4115.020901963108
22 3304.147577608026
23 1871.9906042227644
24 1535.3952520534845
25 1000.168963864047
26 885.0677857850446
27 817.3453896161483
28 776.7515504202973
29 737.7513349977434
30 706.7782576643247
31 679.10267316376
32 654.8675915156981
33 633.1834983202016
34 613.7383346182683
35 595.9714505979456
36 579.6456267410438
37 564.4731024172992
38 550.3153655460038
39 537.0230773235097
40 524.4862627701294
41 512.5998471616883
42 501.28381814161366
43 490.4790893435197
44 480.1429759269939
45 470.2468242191074
46 460.7650542958597
47 451.6788266

527 9.447242110561847
528 9.414455424371688
529 9.381962741594839
530 9.349759814168348
531 9.317842473703154
532 9.286206629868982
533 9.254848268804711
534 9.223763451554326
535 9.192948312528792
536 9.162399057994318
537 9.132111964587134
538 9.102083377854685
539 9.072309710823596
540 9.042787442594335
541 9.013513116962315
542 8.984483341065584
543 8.955694784058856
544 8.927144175813776
545 8.898828305645068
546 8.870744021062508
547 8.842888226548348
548 8.81525788235991
549 8.787850003357162
550 8.760661657854724
551 8.7336899664983
552 8.706932101164824
553 8.680385283886219
554 8.654046785796425
555 8.627913926100957
556 8.601984071069058
557 8.576254633047972
558 8.550723069498499
559 8.525386882052198
560 8.50024361558907
561 8.475290857335997
562 8.450526235985047
563 8.425947420831502
564 8.401552120931129
565 8.377338084276447
566 8.353303096991278
567 8.329444982543711
568 8.30576160097645
569 8.2822508481549
570 8.258910655031833
571 8.23573898692907
572 8.212733842835

In [85]:
value = torch.argmax( torch_activity(DESIGN, PARAM), 1)
hit = (value == TARGET)
acc = torch.mean(hit.float())
print(acc)

tensor(1.)
